# Mapeando el Universo



## Análisis y Curación

 **1 - Análisis general**
 
 * Describir el conjunto de datos. Tratamiento de datos faltantes, outliers, datos repetidos.
 * Considerando el objetivo pricipal de la mentoría, cuáles creen que serían las variables más importantes a considerar?
 * Implementar herramientas en pos de la reproducibilidad de resultados.
 
**2 - Análisis estadístico** 
  * Nos ha contado el especialista que del proceso de adquisición de los mismos se asegura que los datos son correctos y/o completos para aquellos objetos que tienen Magnitudes Pretosian en la banda r entre 14.5 y 17.77. Incluyan esta selección en su análisis.
  * Explorar la base de datos en búsqueda de datos repetidos, en caso de hallarlos elegir la estrategia para su limpieza.
  * Dado que los datos son recolectados a través de mapeos sucesivos del cielo es posible que una galaxia sea observada más de una vez, y en cada observación se le asigne un ID diferente. Propongan una idea para defectar la base de datos de este problema.
       * **Extra**: el especialista en datos no confía en objetos que tengan una separación angular con otro objeto menor a 1" de arco. Intenten limpiar los datos empleando este criterio.
  * Propongan una estrategia para tratar los datos faltantes (en algunas columnas los datos faltantes fueron sustituidos por valores extremos). Quitar o imputar?. Calcular los estadísticos de las distribuciones resultantes.
  * Añadir los campos con los colores calculados tanto para magnitudes petrosian como magnitudes model.
  * En función de las correlaciones observadas en el práctico anterior, y con respecto a nuestra variable objetivo, propongan qué variables se podría excluir del dataset. 
  * Guardar el dataset "limpio" y generar un archivo README que contenga todos los pasos implementados en la curación de los datos.
  * Armen un contenedor de docker con las versiones exactas de todas las bibliotecas utilizadas.

### Lectura de datos

Esto es una manera, pueden utilizar las que más les convenga

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
filename = "DiploDatos.csv"

In [5]:
df = pd.read_csv(filename, index_col=0)
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 537174960955746304 to 647395508997875712
Columns: 115 entries, z to extinction_z
dtypes: float64(113), int64(1), object(1)
memory usage: 4.6 MB


In [6]:
df['modelColor_ug'] = df['modelMag_u'] - df['modelMag_g']
df['modelColor_gr'] = df['modelMag_g'] - df['modelMag_r']
df['modelColor_ri'] = df['modelMag_r'] - df['modelMag_i']
df['modelColor_iz'] = df['modelMag_i'] - df['modelMag_z']
df['petroColor_ug'] = df['petroMag_u'] - df['petroMag_g']
df['petroColor_gr'] = df['petroMag_g'] - df['petroMag_r']
df['petroColor_ri'] = df['petroMag_r'] - df['petroMag_i']
df['petroColor_iz'] = df['petroMag_i'] - df['petroMag_z']

### Uniendo dataframes

Vamos a ir un poco más allá y vamos a unir la tabla anterior con otra donde para algunas de las galaxias la gente a votado si se corresponde con una galaxia espiral, eliptica o irregular.

In [7]:
filename = 'Dataset_Zoo.csv'

In [8]:
zoo = pd.read_csv(filename,index_col=0,dtype = {'spiral':np.int64,'elliptical':np.int64,'uncertain':np.int64})

In [9]:
zoo.dtypes

spiral        int64
elliptical    int64
uncertain     int64
dtype: object

In [10]:
dd = df.join(zoo)
dd.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 537174960955746304 to 647395508997875712
Columns: 126 entries, z to uncertain
dtypes: float64(121), int64(4), object(1)
memory usage: 5.2 MB


In [11]:
dd.head()

,z,zErr,zWarning,subClass,velDisp,velDispErr,ra,dec,raErr,decErr,...,modelColor_gr,modelColor_ri,modelColor_iz,petroColor_ug,petroColor_gr,petroColor_ri,petroColor_iz,spiral,elliptical,uncertain
specObjID,,,,,,,,,,,,,,,,,,,,,
537174960955746304,0.120811,0.000022,0,NaN,217.59020,10.25926,144.493971,2.210449,0.006186,0.005806,...,1.06015,0.46939,0.36341,2.14849,1.02347,0.46319,0.37277,0,0,1
963908897964320768,0.184984,0.000056,0,NaN,216.62740,20.39831,209.490025,5.283101,0.010478,0.010642,...,1.29482,0.48549,0.34711,1.21057,1.32124,0.42321,0.32182,0,0,1
963899002359670784,0.123085,0.000024,0,NaN,125.84820,18.16804,209.532566,5.248963,0.013511,0.018933,...,0.72718,0.40336,0.23817,1.03001,0.63254,0.33633,0.07673,0,0,1
650881510949283840,0.033447,0.000016,0,STARFORMING,69.24703,35.04654,160.169765,5.365856,0.057035,0.051051,...,0.46788,0.30681,0.16218,1.88227,0.29779,0.35605,0.06031,0,0,1
476422542035281920,0.115328,0.000017,0,NaN,81.90747,21.27902,19.808034,15.684047,0.021823,0.012641,...,0.71120,0.45641,0.32606,1.43569,0.63786,0.36858,0.33902,1,0,0


 **1 - Análisis general**

 * Describir el conjunto de datos. Tratamiento de datos faltantes, outliers, datos repetidos.

In [12]:
dd.shape

(5000, 126)

In [13]:
dd.describe()

,z,zErr,zWarning,velDisp,velDispErr,ra,dec,raErr,decErr,modelMag_u,...,modelColor_gr,modelColor_ri,modelColor_iz,petroColor_ug,petroColor_gr,petroColor_ri,petroColor_iz,spiral,elliptical,uncertain
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,0.106732,0.000021,0.016000,136.691611,14.852510,184.204791,25.125391,0.016671,0.016180,19.539480,...,0.861998,0.416104,0.291464,1.621112,0.838083,0.392417,0.243401,0.28880,0.099800,0.611400
std,0.054623,0.000013,0.466248,75.595506,19.427326,57.759337,18.694276,0.041910,0.033692,1.076405,...,0.317552,0.189060,0.224183,0.777162,0.305624,0.226851,0.314662,0.45325,0.299763,0.487481
min,0.010423,0.000003,0.000000,0.000000,-3.000000,0.001718,-11.179560,0.001259,0.001218,15.335180,...,-5.642810,-3.172070,-6.301100,-6.913450,-6.157660,-10.833440,-9.160190,0.00000,0.000000,0.000000
25%,0.066716,0.000011,0.000000,84.571003,9.171935,150.622355,10.275177,0.005682,0.005586,18.939248,...,0.684655,0.376722,0.251500,1.244755,0.649635,0.354535,0.187820,0.00000,0.000000,0.000000
50%,0.100656,0.000019,0.000000,131.921400,12.240650,184.413855,23.384139,0.008408,0.008254,19.595610,...,0.878720,0.424330,0.314960,1.539845,0.847500,0.406400,0.270980,0.00000,0.000000,1.000000
75%,0.138991,0.000028,0.000000,182.166950,16.379272,219.592476,39.191761,0.015174,0.015143,20.245438,...,1.034527,0.467097,0.355032,1.817260,1.006263,0.449853,0.326592,1.00000,0.000000,1.000000
max,0.746932,0.000305,16.000000,850.000000,1131.371000,359.995051,69.241324,1.817576,0.789803,28.112870,...,6.407680,7.526770,4.397010,16.659360,6.369250,6.933700,10.373380,1.00000,1.000000,1.000000


In [26]:
def total_missings_table(dataset):
    variables = []
    missings = []
    relevance = []
    for i in list(dataset.columns):
        variables.append(i)
        missings.append(dataset[i].isnull().sum())
        relevance.append(len(dataset[i].dropna())/len(dataset[i]))
    table = pd.DataFrame(data = missings, index = variables)
    table.columns = ['Total missing Values']
    return(table)
total_missings_table(dd)

,Total missing Values
z,0
zErr,0
zWarning,0
subClass,2929
velDisp,0
...,...
petroColor_ri,0
petroColor_iz,0
spiral,0
elliptical,0


In [15]:
def outliers(dataset, column):
    outliers_column = dataset[dataset[column] > (dataset[column].mean + 3 * dataset[column].std()) ]
    return(outliers_column)

In [16]:
dd[dd.index.duplicated(keep=False)]

,z,zErr,zWarning,subClass,velDisp,velDispErr,ra,dec,raErr,decErr,...,modelColor_gr,modelColor_ri,modelColor_iz,petroColor_ug,petroColor_gr,petroColor_ri,petroColor_iz,spiral,elliptical,uncertain
specObjID,,,,,,,,,,,,,,,,,,,,,


No hay índices duplicados

 * Considerando el objetivo pricipal de la mentoría, cuáles creen que serían las variables más importantes a considerar?
 * Implementar herramientas en pos de la reproducibilidad de resultados.

**2 - Análisis estadístico** 
  * Nos ha contado el especialista que del proceso de adquisición de los mismos se asegura que los datos son correctos y/o completos para aquellos objetos que tienen Magnitudes Pretosian en la banda r entre 14.5 y 17.77. Incluyan esta selección en su análisis.

In [54]:
dd = dd[(dd.petroMag_r>=14.5) & (dd.petroMag_r<=17.77)]

 * Explorar la base de datos en búsqueda de datos repetidos, en caso de hallarlos elegir la estrategia para su limpieza.

 * Dado que los datos son recolectados a través de mapeos sucesivos del cielo es posible que una galaxia sea observada más de una vez, y en cada observación se le asigne un ID diferente. Propongan una idea para defectar la base de datos de este problema.
     * **Extra**: el especialista en datos no confía en objetos que tengan una separación angular con otro objeto menor a 1" de arco. Intenten limpiar los datos empleando este criterio.

Una forma para detectar si una galaxia ha sido observada más de una vez puede ser buscar valores repetidos en las variables de interés. Sin embargo, esto puede no ser la mejor estrategia porque cada variable de interés tiene asociada una medición de error. Por lo tanto, es posible que esta medición de error se esté llevando parte de la variable de interés provocando que una variable para la misma galaxia no sea exactamente igual. En este caso, con esta técnica podríamos "subdetectar" galaxias medidas más de una vez.
Por lo tanto, vamos a detectar las galaxias medidas más de una vez desconfiando de aquellos objetos que tengan una separación angular con otro objeto menor a 1" de arco. 

Fórmula Separación Angular

${\displaystyle \theta =\arccos {\Big [}\sin(\delta _{1})\sin(\delta _{2})+\cos(\delta _{1})\cos(\delta _{2})\cos(\alpha _{1}-\alpha _{2}){\Big ]}}$

donde ${\displaystyle \alpha }$ son ascensiones rectas (ra) y ${\displaystyle \delta }$ son las declinaciones (dec) (Coordenadas ecuatoriales) expresados en ángulos de la forma ${\displaystyle \alpha \in [0,2\pi ]}$ y ${\displaystyle \delta \in [-\pi /2,\pi /2]}$ 

* 1) Forma más eficiente

In [62]:
import itertools
indexes = list(itertools.combinations(list(dd.index),2))
tuples = []
for i,j in indexes:
    ra_i = float(dd.ra[dd.index == i]) 
    ra_j = float(dd.ra[dd.index == j])
    dec_i = float(dd.dec[dd.index == i])
    dec_j = float(dd.dec[dd.index == j])
    theta = np.arccos( np.sin(dec_i) * np.sin(dec_j) + np.cos(dec_i)*np.cos(dec_j)*np.cos(ra_i - ra_j))
    if (1/3600)*theta < 1:
        tuples.append((i,j))
    else: continue
tuples = []

MemoryError: 

In [ ]:
repeated = []
for i,j in tuples:
    if j not in repeated:
        repeated.append(j)
    else: continue
repeated

In [ ]:
dd = dd.drop(index = repeated)

* 2) Forma menos eficiente (armando un grafo)

In [42]:
nodos = []
for i in dd.index:
    for j in dd.index:
        if i == j:
            continue
        else:
            ra_i = float(dd.ra[dd.index == i]) 
            ra_j = float(dd.ra[dd.index == j])
            dec_i = float(dd.dec[dd.index == i])
            dec_j = float(dd.dec[dd.index == j])
            theta = np.arccos( np.sin(dec_i)*np.sin(dec_j) + np.cos(dec_i)*np.cos(dec_j)*np.cos(ra_i - ra_j))
            if (1/3600)*theta < 1:
                nodos.append((i,j))
            else: 
                continue
nodos = []

KeyboardInterrupt: 

In [93]:
grafo = dict()
for x,y in nodos:
    grafo.setdefault(x, []).append(y)
len(grafo)

37